In [1]:
import requests
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import openpyxl
import os
from dotenv import load_dotenv



In [2]:
# Load the environment variables
load_dotenv()

# API key
map_api_key = os.getenv('GOOGLE_MAPS_API_KEY')
sheet_api_key = os.getenv('GOOGLE_SHEET_API_KEY')


In [ ]:

def get_coordinates_google(address, map_api_key):
    try:
        # Google Maps Geocoding API endpoint
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={map_api_key}"
        response = requests.get(url)
        data = response.json()

        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return {
                "address": address,
                "latitude": location['lat'],
                "longitude": location['lng']
            }
        else:
            return {
                "address": address,
                "latitude": None,
                "longitude": None,
                "status": data['status']
            }
    except Exception as e:
        return {
            "address": address,
            "latitude": None,
            "longitude": None,
            "error": str(e)
        }

# List of addresses
addresses = [
    "Wellcome Trust Genome Campus, Hinxton, Saffron Walden CB10 1RQ, United Kingdom",
    "2451 Signal St, San Pedro, CA 90731",
    "University of Canberra, Bruce ACT 2617 Australia"
]

# Get coordinates for each address
results = [get_coordinates_google(address, map_api_key) for address in addresses]

# Display results
for result in results:
    print(f"Address: {result['address']}")
    print(f"Latitude: {result['latitude']}\nLongitude: {result['longitude']}")
    if "status" in result:
        print(f"Status: {result['status']}")
    print()


Address: Wellcome Trust Genome Campus, Hinxton, Saffron Walden CB10 1RQ, United Kingdom
Latitude: 52.0810484
Longitude: 0.1838827

Address: 2451 Signal St, San Pedro, CA 90731
Latitude: 33.7222724
Longitude: -118.2734459

Address: University of Canberra, Bruce ACT 2617 Australia
Latitude: -35.2381421
Longitude: 149.0838384



In [3]:
sheet_url = "https://docs.google.com/spreadsheets/d/1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE/edit?gid=0#gid=0"

# Test sheet accessibility
sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
test_url = f"https://sheets.googleapis.com/v4/spreadsheets/{sheet_id}?key={sheet_api_key}"
response = requests.get(test_url)
print("Response status:", response.status_code)
print("Response:", response.text[:200])  # Print first 200 characters of response

Response status: 200
Response: {
  "spreadsheetId": "1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE",
  "properties": {
    "title": "Master NETWORK OF NETWORKS MAP SHEET",
    "locale": "en_US",
    "autoRecalc": "ON_CHANGE",
    "t


In [9]:
def test_sheet_access(sheet_url, sheet_api_key):
    try:
        # Extract sheet ID from URL
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
        
        # Test 1: Basic API Access
        print("Test 1: Testing basic API access...")
        test_url = f"https://sheets.googleapis.com/v4/spreadsheets/{sheet_id}?key={sheet_api_key}"
        response = requests.get(test_url)
        print(f"API Response Status: {response.status_code}")
        
        if response.status_code != 200:
            print("❌ Failed to access the sheet via API")
            return False
            
        # Test 2: Get Sheet Metadata
        print("\nTest 2: Retrieving sheet metadata...")
        sheet_data = response.json()
        print(f"Sheet Title: {sheet_data['properties']['title']}")
        print(f"Sheet ID: {sheet_data['spreadsheetId']}")
        
        # Test 3: Try to read actual data with a smaller range
        print("\nTest 3: Attempting to read sheet data...")
        values_url = f"https://sheets.googleapis.com/v4/spreadsheets/{sheet_id}/values/A1:A1?key={sheet_api_key}"
        values_response = requests.get(values_url)
        
        if values_response.status_code == 200:
            values_data = values_response.json()
            if 'values' in values_data:
                print("\nFirst cell of data:")
                print(values_data['values'])
                print("\n✅ Successfully read sheet data!")
            else:
                print("❌ No data found in the sheet")
        else:
            print(f"❌ Failed to read sheet data. Status code: {values_response.status_code}")
            print(f"Error details: {values_response.text}")
            
        return True
        
    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")
        return False

# Execute the test
print("Starting sheet accessibility test...")
sheet_url = "https://docs.google.com/spreadsheets/d/1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE/edit?gid=0#gid=0"
sheet_api_key = os.getenv('GOOGLE_SHEET_API_KEY')
test_sheet_access(sheet_url, sheet_api_key)

Starting sheet accessibility test...
Test 1: Testing basic API access...
API Response Status: 200

Test 2: Retrieving sheet metadata...
Sheet Title: Master NETWORK OF NETWORKS MAP SHEET
Sheet ID: 1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE

Test 3: Attempting to read sheet data...

First cell of data:
[['Affiliated Project']]

✅ Successfully read sheet data!


True

In [14]:
def inspect_sheet_structure(sheet_url, sheet_api_key):
    """Inspect the structure of the Google Sheet."""
    try:
        # Extract sheet ID from URL
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
        
        # Create service
        service = build('sheets', 'v4', developerKey=sheet_api_key)
        
        # Get the first sheet's data
        sheet = service.spreadsheets()
        result = sheet.values().get(
            spreadsheetId=sheet_id,
            range='A:Z'  # Get all columns
        ).execute()
        
        values = result.get('values', [])
        
        if values:
            print("\nHeaders:")
            print(values[0])
            
            if len(values) > 1:
                print("\nFirst row of data:")
                print(values[1])
        else:
            print("No data found in sheet")
            
    except Exception as e:
        print(f"Error inspecting sheet: {str(e)}")
        raise

# Execute the inspection
print("Starting sheet structure inspection...")
sheet_url = "https://docs.google.com/spreadsheets/d/1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE/edit?gid=0#gid=0"
sheet_api_key = os.getenv('GOOGLE_SHEET_API_KEY')
inspect_sheet_structure(sheet_url, sheet_api_key)

Starting sheet structure inspection...

Headers:
['Affiliated Project', 'Title', 'Map Differentiation', 'Activities Occurring', 'Address']

First row of data:
['Canadian BioGenome Project (CANBP)', 'BC Cancer', 'Headquarters', 'DNA Sequencing (long reads), DNA Sequencing (short reads), Bioinformatics (assembly), Bioinformatics (annotation)', '675 W 10th Ave, Vancouver, BC V5Z 0B4, Canada']


In [ ]:
def process_google_sheet(sheet_url, map_api_key, sheet_api_key):
    """Process all sheets in a Google Sheets file, add latitude and longitude, and combine into one Excel workbook."""
    try:
        # Retrieve the Google Sheet ID from the URL
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]

        # Retrieve all sheet GIDs and names dynamically
        sheet_metadata = get_sheet_metadata(sheet_id, sheet_api_key)

        # Base export URL for multiple sheets
        base_csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid="

        # Initialize an Excel writer
        output_excel_file = "affiliate_network_list.xlsx"
        writer = pd.ExcelWriter(output_excel_file, engine='openpyxl')

        for gid, sheet_name in sheet_metadata.items():
            print(f"\nProcessing sheet: {sheet_name}")
            
            # Generate the CSV URL for the current sheet
            csv_url = f"{base_csv_url}{gid}"

            # Read the current sheet into a DataFrame
            data = pd.read_csv(csv_url)
            
            # Verify the Address column exists
            if 'Address' not in data.columns:
                print(f"Warning: No 'Address' column found in sheet '{sheet_name}'. Skipping...")
                continue

            # Add latitude and longitude columns
            data['Latitude'] = None
            data['Longitude'] = None

            # Process each row
            total_rows = len(data)
            print(f"Processing {total_rows} rows...")

            # Loop through each address and fetch coordinates
            for index, row in data.iterrows():
                address = row['Address']
                if pd.isna(address):
                    print(f"Skipping row {index + 1}: No address found")
                    continue
                    
                print(f"\nProcessing row {index + 1}/{total_rows}")
                print(f"Address: {address}")
                
                latitude, longitude = get_coordinates(address, map_api_key)
                data.at[index, 'Latitude'] = latitude
                data.at[index, 'Longitude'] = longitude
                
                print(f"Coordinates: {latitude}, {longitude}")

            # Write the DataFrame to a sheet in the Excel workbook
            data.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"✅ Processed sheet '{sheet_name}' and added it to the Excel workbook.")

        # Save the combined Excel workbook
        writer.close()
        print(f"\n✅ All sheets have been combined and saved to '{output_excel_file}'.")

    except Exception as e:
        print(f"Error processing sheet: {str(e)}")
        raise

# Process the Google Sheet
print("Starting sheet processing...")
process_google_sheet(sheet_url, map_api_key, sheet_api_key)

In [10]:
# fetch data from publicly accessible google sheet
def get_coordinates(address, map_api_key):
    """Fetch coordinates for a given address using Google Maps Geocoding API."""
    if not address:
        print("Address is empty or invalid.")
        return None, None

    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address.replace(' ', '+')}&key={map_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Failed to geocode address: {address} | Status: {data['status']}")
            return None, None
    else:
        print(f"API request error: {response.status_code}")
        return None, None

def get_sheet_metadata(sheet_id, sheet_api_key):
    """Retrieve metadata of all sheets in the Google Sheets file."""
    try:
        # Create credentials object with just the API key
        from google.oauth2.credentials import Credentials
        from google.auth.credentials import AnonymousCredentials
        
        # Build the service with anonymous credentials and API key
        service = build(
            'sheets', 
            'v4',
            credentials=AnonymousCredentials(),
            developerKey=sheet_api_key
        )
        
        # Get the spreadsheet metadata
        sheet_metadata = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        return {sheet['properties']['sheetId']: sheet['properties']['title'] for sheet in sheets}
    except HttpError as err:
        print(f"Error details: {err}")
        if "403" in str(err):
            print("\nThis might mean either:")
            print("1. The Google Sheet is not publicly accessible")
            print("2. The API key doesn't have access to Google Sheets API")
            print("\nPlease check:")
            print("1. Share settings of your Google Sheet (make it 'Anyone with the link can view')")
            print("2. Enable Google Sheets API in Google Cloud Console")
        raise Exception(f"Error fetching sheet metadata: {err}")


def process_google_sheet(sheet_url, map_api_key, sheet_api_key):
    """Process all sheets in a Google Sheets file, add latitude and longitude, and combine into one Excel workbook."""
    # Retrieve the Google Sheet ID from the URL
    sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]

    # Retrieve all sheet GIDs and names dynamically
    sheet_metadata = get_sheet_metadata(sheet_id, sheet_api_key)

    # Base export URL for multiple sheets
    base_csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid="

    # Initialize an Excel writer
    output_excel_file = "affiliate_network_list.xlsx"
    writer = pd.ExcelWriter(output_excel_file, engine='openpyxl')

    for gid, sheet_name in sheet_metadata.items():
        # Generate the CSV URL for the current sheet
        csv_url = f"{base_csv_url}{gid}"

        # Read the current sheet into a DataFrame, skipping the first row
        data = pd.read_csv(csv_url, skiprows=1)

        # Add latitude and longitude columns
        data['Latitude'] = None
        data['Longitude'] = None

        # Loop through each address and fetch coordinates
        for index, row in data.iterrows():
            address = row['Address']  # Assuming addresses are in column D
            latitude, longitude = get_coordinates(address, map_api_key)
            data.at[index, 'Latitude'] = latitude
            data.at[index, 'Longitude'] = longitude
            print(f"Address: {address} | Latitude: {latitude}, Longitude: {longitude}")
        

        # Write the DataFrame to a sheet in the Excel workbook
        data.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Processed sheet '{sheet_name}' and added it to the Excel workbook.")

    # Save the combined Excel workbook
    # writer.close()
    # print(f"All sheets have been combined and saved to '{output_excel_file}'.")



In [11]:

# Process the Google Sheet
process_google_sheet(sheet_url, map_api_key, sheet_api_key)

KeyError: 'Address'